In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

pd.set_option('display.max_columns', 144)

In [2]:
# Data irradiance and temperature taken from Nrel: NSRDB, this numbers represents the geolocalization
data = pd.read_csv(r'..\dataset\NSRDB_Ijui_RS_2020_8362102_-28.38_-53.93_2020.csv', header=2, delimiter=',')
data['GHI'] = data['GHI']/1000
# take the mean and standard deviation per month from the 144pts
irradiance = pd.pivot_table(data, index=['Month'], columns=['Hour', 'Minute'], values=['GHI'], aggfunc=[np.mean, np.std])
temperature = pd.pivot_table(data, index=['Month'], columns=['Hour', 'Minute'], values=['Temperature'], aggfunc=[np.mean])


In [3]:
# Select the temperature and irradiance from January
temp_medjan = temperature['mean'].loc[1].values
irrad_medjan = irradiance['mean'].loc[1].values
irrad_stdjan = irradiance['std'].loc[1].values

In [4]:
# Calculating alfa and beta that represents one day of January
beta_jan = np.zeros((np.shape(irrad_medjan)))
alpha_jan = np.zeros((np.shape(irrad_medjan)))
for i in range(len(irrad_stdjan)):
    if irrad_stdjan[i] == 0:
        beta_jan[i] = 0
        alpha_jan[i] = 0
    else:
        beta_jan[i] = (1-irrad_medjan[i])*((irrad_medjan[i]*(1-irrad_medjan[i]))/(irrad_stdjan[i]**2) - 1)
        alpha_jan[i] = (irrad_medjan[i]*beta_jan[i])/(1-irrad_medjan[i])

In [5]:
# Number of samples
N = 1000 

# Initialize a random generator
rng = np.random.default_rng(15)

# Initialize the arrays of irradiance curves
irrad_summer = np.zeros((N, len(irrad_stdjan)))

# Generate N samples of 144 timesteps (1 day)
for i in range(len(irrad_stdjan)):
    if (alpha_jan[i] or beta_jan[i]) <= 0:
        irrad_summer[:,i] = 0
    else:
        irrad_summer[:,i] = rng.beta(alpha_jan[i], beta_jan[i], N) 

In [6]:
# Data of the PV system 
pmpref = 270  # in (Wp)
noct   = 45   # Nominal operating cell temperature (Datasheet)
n      = 8   # Number of modules (2.1kWp)
ninv   = 0.95 # Inverter efficiency

# From kW/m2 to W/m2
irrad_summer = irrad_summer*1000

# Initializing arrays
tc_s = np.zeros((N, len(irrad_stdjan)))
pmp_s = np.zeros((N, len(irrad_stdjan)))
pv_s = np.zeros((N, len(irrad_stdjan)))

# Calculating the power generated by the PVs
for i in range(N):
    for j in range(len(irrad_stdjan)):
        tc_s[i,j] = temp_medjan[j]+((noct-20)/800)*irrad_summer[i,j]
        pmp_s[i,j] = n*pmpref*(irrad_summer[i,j]/1000)*(1-0.0043*(tc_s[i,j]-25))
        pv_s[i,j] = (pmp_s[i,j]*ninv)*10**-3 # Potencia de saída da usina Fotovoltaica (em kW)

# From kW to MW
# PV_s = pv_s/1000

date = datetime.now().strftime('%d_%m-%H_%M')
# Save the fotovoltaic curves
np.savez(f"../dataset/PV_Curves_Ijui_RS_{N}_Samples_{date}", PV_s=pv_s)